<a href="https://colab.research.google.com/github/RajeswariKumaran/SSLMethodsAnalysis/blob/main/EqualHyperParameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 22.0 MB/s eta 0:00:00


In [ ]:
# starting here

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import optuna

In [ ]:
# load the datasets and get labeled subset for fully supervised model
# Dataset loading (CIFAR-10)
# Dataset loading (CIFAR-10)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split into labeled and unlabeled sets
labeled_data_size = 6000  # Assume we have 4000 labeled samples
indices = torch.randperm(len(trainset)).tolist()
labeled_indices = indices[:labeled_data_size]
unlabeled_indices = indices[labeled_data_size:]

labeled_trainset = Subset(trainset, labeled_indices)
unlabeled_trainset = Subset(trainset, unlabeled_indices)

# Data loaders
batch_size = 64
labeled_trainloader = DataLoader(labeled_trainset, batch_size=batch_size, shuffle=True)
unlabeled_trainloader = DataLoader(unlabeled_trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

from torch.utils.data import random_split

# Set the proportion of validation data (e.g., 20%)
val_split_ratio = 0.2

# Compute lengths for training and validation sets
total_size = len(labeled_trainset)
val_size = int(total_size * val_split_ratio)
train_size = total_size - val_size

# Perform the split
labeled_trainset, validset = random_split(labeled_trainset, [train_size, val_size])

# Now you can create DataLoaders
labeled_trainloader = DataLoader(labeled_trainset, batch_size=64, shuffle=True)
labeled_validloader = DataLoader(validset, batch_size=64, shuffle=False)


In [ ]:
# optimize hyper parameters for supervised training with only labeled data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import optuna  # Hyperparameter optimization library

# Define the same model as used for SSL (for a fair comparison)
class SupervisedModel(nn.Module):
    def __init__(self):
        super(SupervisedModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

# Define the objective function for Optuna optimization
def objective_supervised(trial):
    # Hyperparameter search space
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)  # Log scale for learning rate
    momentum = trial.suggest_uniform('momentum', 0.5, 0.9)  # Momentum for SGD
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Different batch sizes

    # Create the DataLoader for labeled data
    labeled_trainloader = DataLoader(labeled_trainset, batch_size=batch_size, shuffle=True)

    # Initialize the supervised model, optimizer, and loss criterion
    model = SupervisedModel()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    criterion = nn.CrossEntropyLoss()

    # Train the model with the current hyperparameters
    model.train()
    for epoch in range(10):  # Training for 10 epochs
        running_loss = 0.0
        for inputs, labels in labeled_trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {running_loss/len(labeled_trainloader)}")

    # Evaluate the model on the validation set after training
    accuracy = evaluate_supervised_model(model, validloader)  # Assuming validloader is predefined
    return accuracy  # Minimize the negative accuracy (maximize accuracy)

# Evaluation function
def evaluate_supervised_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Create validation set (same as test set or separate split from training data)
validloader = DataLoader(validset, batch_size=64, shuffle=False)  # Assuming validset is predefined

# Set up Optuna study to find the best hyperparameters
study = optuna.create_study(direction='maximize')  # We want to maximize accuracy
study.optimize(objective_supervised, n_trials=20)  # Search for 20 trials

# Print best hyperparameters and accuracy
print(f"Best trial: {study.best_trial.params}")
print(f"Best validation accuracy: {study.best_trial.value}%")



[I 2025-08-31 06:38:17,877] A new study created in memory with name: no-name-da8cf57e-c808-40d0-8344-bea00303c0c5
/tmp/ipython-input-940294220.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)  # Log scale for learning rate
/tmp/ipython-input-940294220.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  momentum = trial.suggest_uniform('momentum', 0.5, 0.9)  # Momentum for SGD
[I 2025-08-31 06:38:46,982] Trial 0 finished with value: 29.75 and parameters: {'lr': 5.9321778995265514e-05, 'momentum': 0.6861471040931886, 'batch_size': 32}. Best is trial 0 with value: 29.75.
[I 2025-08-31 06:39:16,324] Trial 1 finished with value: 

Best trial: {'lr': 0.007368715747923294, 'momentum': 0.8974213852548178, 'batch_size': 32}
Best validation accuracy: 51.333333333333336%


In [ ]:
supervised_best_params = study.best_trial.params

In [ ]:
# get best parameters for semi supervised learning using VAT
import optuna
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Define your VAT model (as in your original code)
class VATModel(nn.Module):
    def __init__(self):
        super(VATModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

# Define VAT loss function
def virtual_adversarial_loss(model, x, epsilon=1e-6):
    x.requires_grad_()
    logits = model(x)

    # Virtual adversarial perturbation
    loss = F.cross_entropy(logits, torch.max(logits, 1)[1])
    loss.backward()
    grad = x.grad

    # Perturbation
    perturbation = epsilon * torch.sign(grad)

    # Perturbed data
    x_perturbed = x + perturbation
    logits_perturbed = model(x_perturbed)
    loss_perturbed = F.cross_entropy(logits_perturbed, torch.max(logits, 1)[1])

    return loss_perturbed

# Combined loss for VAT model
def vat_loss(model, x, labels, criterion, epsilon=1e-6, alpha=1.0):
    # Cross-entropy loss for labeled data
    ce_loss = criterion(model(x), labels)

    # Virtual adversarial loss (VAT)
    va_loss = virtual_adversarial_loss(model, x, epsilon)

    return ce_loss + alpha * va_loss

# Training function for VAT model
def train_vat_model(model, labeled_trainloader, unlabeled_trainloader, criterion, optimizer, num_epochs=10, epsilon=1e-6, alpha=1.0):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for (inputs, labels), (inputs_unlabeled, _) in zip(labeled_trainloader, unlabeled_trainloader):
            optimizer.zero_grad()
            # Train on labeled data with VAT applied to unlabeled data
            loss = vat_loss(model, inputs, labels, criterion, epsilon, alpha)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(labeled_trainloader)}")

# Optuna objective function for hyperparameter search
def optimize_vat_hyperparameters(labeled_trainloader, unlabeled_trainloader, testloader):
    def objective(trial):
        # Suggest hyperparameters
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
        momentum = trial.suggest_uniform('momentum', 0.8, 0.95)
        epsilon = trial.suggest_loguniform('epsilon', 1e-6, 1e-1)
        alpha = trial.suggest_uniform('alpha', 0.5, 2.0)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

        # Create model, optimizer, and criterion
        model = VATModel()
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        criterion = nn.CrossEntropyLoss()

        # DataLoader for current batch size
        labeled_trainloader = DataLoader(labeled_trainset, batch_size=batch_size, shuffle=True)
        unlabeled_trainloader = DataLoader(unlabeled_trainset, batch_size=batch_size, shuffle=True)

        # Train the VAT model
        train_vat_model(model, labeled_trainloader, unlabeled_trainloader, criterion, optimizer, num_epochs=10, epsilon=epsilon, alpha=alpha)

        # Evaluate the model after training
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in validloader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy  # Maximize accuracy

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    print(f"Best trial: {study.best_trial.value}")
    print(f"Best hyperparameters: {study.best_trial.params}")
    return study.best_trial.params

# Run hyperparameter optimization
vat_best_params = optimize_vat_hyperparameters(labeled_trainloader, unlabeled_trainloader, testloader)

# Now you can use `vat_best_params` to train the VAT model with optimal hyperparameters.

[I 2025-08-31 07:41:36,044] A new study created in memory with name: no-name-e9ff834d-e65e-4743-a307-4802fc04eff0
/tmp/ipython-input-772745886.py:71: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/tmp/ipython-input-772745886.py:72: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  momentum = trial.suggest_uniform('momentum', 0.8, 0.95)
/tmp/ipython-input-772745886.py:73: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  epsilon = trial.suggest_loguniform('epsilon', 1e-6, 1e-1)
/tmp/ipython-

Epoch 1, Loss: 3.503390174163015
Epoch 2, Loss: 3.182346594961066
Epoch 3, Loss: 3.1231669187545776
Epoch 4, Loss: 3.065893813183433
Epoch 5, Loss: 3.0262865392785323
Epoch 6, Loss: 2.9909278781790483
Epoch 7, Loss: 2.964064880421287
Epoch 8, Loss: 2.940559048401682
Epoch 9, Loss: 2.916605886660124
Epoch 10, Loss: 2.903702735900879


[I 2025-08-31 07:43:07,514] Trial 0 finished with value: 11.833333333333334 and parameters: {'lr': 0.0002933545621957897, 'momentum': 0.8726100152878178, 'epsilon': 2.2241271650750257e-06, 'alpha': 0.6242247997400614, 'batch_size': 128}. Best is trial 0 with value: 11.833333333333334.


Epoch 1, Loss: 3.9939275328318278
Epoch 2, Loss: 3.320546964009603
Epoch 3, Loss: 3.1655237038930255
Epoch 4, Loss: 3.0780448627471926
Epoch 5, Loss: 2.9674384212493896
Epoch 6, Loss: 2.819689718882243
Epoch 7, Loss: 2.3940446472167967
Epoch 8, Loss: 2.1679722690582275
Epoch 9, Loss: 2.064407498041789
Epoch 10, Loss: 1.9316555070877075


[I 2025-08-31 07:44:38,457] Trial 1 finished with value: 48.75 and parameters: {'lr': 0.0055408037667521, 'momentum': 0.9106519230578937, 'epsilon': 0.0002303768254359234, 'alpha': 1.8719397650367253, 'batch_size': 64}. Best is trial 1 with value: 48.75.


...

Epoch 1, Loss: 3.4485055764516193
Epoch 2, Loss: 2.984088484446208
Epoch 3, Loss: 2.8282949924468994
Epoch 4, Loss: 2.4647935485839843
Epoch 5, Loss: 2.180817084312439
Epoch 6, Loss: 2.03221075852712
Epoch 7, Loss: 1.8462496916453044
Epoch 8, Loss: 1.778871579170227
Epoch 9, Loss: 1.6450654109319052
Epoch 10, Loss: 1.5741627661387125


[I 2025-08-31 08:10:32,782] Trial 18 finished with value: 48.0 and parameters: {'lr': 0.006093240840507661, 'momentum': 0.8888334970311471, 'epsilon': 2.396935059500899e-05, 'alpha': 1.077791455233288, 'batch_size': 64}. Best is trial 12 with value: 49.75.


Epoch 1, Loss: 3.949950205485026
Epoch 2, Loss: 3.4121944363911947
Epoch 3, Loss: 3.3067706489562987
Epoch 4, Loss: 3.2310439205169676
Epoch 5, Loss: 3.176257158915202
Epoch 6, Loss: 3.1254377714792887
Epoch 7, Loss: 3.0729797649383546
Epoch 8, Loss: 3.034290132522583
Epoch 9, Loss: 2.9915740426381427
Epoch 10, Loss: 2.9537227598826092


[I 2025-08-31 08:12:03,079] Trial 19 finished with value: 10.666666666666666 and parameters: {'lr': 0.0013267130301124423, 'momentum': 0.9189905399254423, 'epsilon': 0.0006977728075051315, 'alpha': 1.799684167772557, 'batch_size': 64}. Best is trial 12 with value: 49.75.


Best trial: 49.75
Best hyperparameters: {'lr': 0.004683444236568706, 'momentum': 0.8915845187796497, 'epsilon': 7.633183703505233e-05, 'alpha': 1.6675574886555689, 'batch_size': 64}


In [ ]:
# Now compare the performance of VAT and fully supervised using respective best parameters

# Initialize and train the fully-supervised model using best parameters
# Define Fully-Supervised Model (Simple CNN)
class FullySupervisedCNN(nn.Module):
    def __init__(self):
        super(FullySupervisedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

model = FullySupervisedCNN()
optimizer = optim.SGD(model.parameters(), lr=supervised_best_params['lr'], momentum=supervised_best_params['momentum'])
criterion = nn.CrossEntropyLoss()

# Training function for models
def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

train_model(model, trainloader, criterion, optimizer, num_epochs=10)

Epoch 1, Loss: 1.4652755618705164
Epoch 2, Loss: 1.1664301307152605
Epoch 3, Loss: 1.047432781332899
Epoch 4, Loss: 0.9822487318912125
Epoch 5, Loss: 0.9283737188105083
Epoch 6, Loss: 0.8880558254392555
Epoch 7, Loss: 0.8585296645951088
Epoch 8, Loss: 0.827574706710208
Epoch 9, Loss: 0.806894422301551
Epoch 10, Loss: 0.7838642739731333


In [ ]:

ssl_model = VATModel()
optimizer_ssl = optim.SGD(ssl_model.parameters(), lr=vat_best_params['lr'], momentum=vat_best_params['momentum'])

# Define criterion for VAT model
criterion_ssl = nn.CrossEntropyLoss()

# Training function for VAT model
def train_vat_model(model, labeled_trainloader, unlabeled_trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for (inputs, labels), (inputs_unlabeled, _) in zip(labeled_trainloader, unlabeled_trainloader):
            optimizer.zero_grad()
            # Train on labeled data with VAT applied to unlabeled data
            loss = vat_loss(model, inputs, labels, criterion, alpha=1.0, epsilon=1e-6)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(labeled_trainloader)}")

# Train SSL (VAT) model

train_vat_model(ssl_model, labeled_trainloader, unlabeled_trainloader, criterion_ssl, optimizer_ssl, num_epochs=10)

Epoch 1, Loss: 3.3468048350016275
Epoch 2, Loss: 2.9587280813852948
Epoch 3, Loss: 2.8115629291534425
Epoch 4, Loss: 2.688424596786499
Epoch 5, Loss: 2.3976821835835773
Epoch 6, Loss: 2.1479301420847574
Epoch 7, Loss: 1.9474782896041871
Epoch 8, Loss: 1.797435245513916
Epoch 9, Loss: 1.7121193997065227
Epoch 10, Loss: 1.6093256950378418


In [ ]:
# Evaluate function
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# ssl_model_accuracy = evaluate(ssl_model)
# print(f"SSL (VAT) model accuracy: {ssl_model_accuracy}%")

In [ ]:
# Compare performance of SSL (VAT) vs Fully-supervised
fully_supervised_accuracy = evaluate(model)
ssl_model_accuracy = evaluate(ssl_model)

print(f"Fully-supervised model accuracy: {fully_supervised_accuracy}%")
print(f"SSL (VAT) model accuracy: {ssl_model_accuracy}%")

# Justifying the claim: Narrow performance gap under equal hyperparameter tuning
if fully_supervised_accuracy >= ssl_model_accuracy:
    print("The performance gap between SSL (VAT) and fully-supervised methods is narrower when hyperparameters are optimally tuned.")
else:
    print("SSL (VAT) methods outperform fully-supervised methods, even with optimal tuning.")

Fully-supervised model accuracy: 63.22%
SSL (VAT) model accuracy: 49.46%
The performance gap between SSL (VAT) and fully-supervised methods is narrower when hyperparameters are optimally tuned.
